# Group-wise operation

This notebook demostrates common group-wise operations by using SDK.

## Prerequisites

You need

- an SDK environment set up with the administrator priviledge. Check *environment setup.ipynb* for the detail.

**Replace the value below with your PrimeHub domain (*http* or *https*).**

In [ ]:
PRIMEHUB_CLUSTER = 'https://c.demo.primehub.io'

**Check if the configuration exists. If not, you need to login to retrieve the token and input it in the prompt. The configuration will be generated at**`~/.primehub/config.json`.

In [ ]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print(f"PrimeHub Python SDK {ph.version.version()} environment is ready, you are good to go.")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("Failed to retrieve the information from PrimeHub cluster, please check the configuration.")

## ToC

- [Fundamental Operations (CRUD)](#fundamental)
- [Common Operations](#common)

## Fundamental Operations <a class="anchor" id="fundamental"></a>

### Create Group

Method: `ph.admin.groups.create(config)`

By using `create()` with a configuration to :
- create a group with preset requirements

Before creating a user, let's decide which group the user belongs to. Check your PrimeHub and replace the `group_name` with yours.

In [ ]:
group_name = 'Continental'

Let's prepare the configuration of creating a group, *Continental*, with the preset requirements.

In [ ]:
# Prepare a group configuration
config = {
  "name": group_name,
  "displayName": "a group of hitman",
  "enabledDeployment": False,
  "enabledSharedVolume": False,
  "quotaCpu": 4,
  "quotaGpu": 4,
  "admins": "",
  "users": {
      "connect": []
  }
}

In [ ]:
specified_group = ph.admin.groups.create(config)
specified_group

### Add/Remove Group Member

Methods
- `ph.admin.groups.connect_user(group_id, user_id`)
- `ph.admin.groups.disconnect_user(group_id, user_id)`

In [ ]:
# Get the user id by the given user name
user_name = 'john-wick'
users = ph.admin.users.list()
for user in users:
    if user['username'] == user_name:
        specified_user = user
specified_user

In [ ]:
# Add group member
ph.admin.groups.connect_user(specified_group['id'], specified_user['id'])

# verify
ph.admin.groups.get(specified_group['id'])['users']

In [ ]:
# remove group member
ph.admin.groups.disconnect_user(specified_group['id'], specified_user['id'])

# verify
ph.admin.groups.get(specified_group['id'])['users']

### Add/Remove Image

Methods:
- `ph.admin.groups.connect_image('group_id','image_id')`
- `ph.admin.groups.disconnect_image('group_id','image_id')`

In [ ]:
# Replace the image's id below with yours
specified_image_id = 'ngc-pytorch-jupyter'

# Connect an image to the group
ph.admin.groups.connect_image(specified_group['id'], specified_image_id)

# verify
ph.admin.groups.get(specified_group['id'])['images']

In [ ]:
# Disconnect an image to the group
ph.admin.groups.disconnect_image(specified_group['id'], specified_image_id)

# verify
ph.admin.groups.get(specified_group['id'])['images']

### Add/Remove InstaceType

Methods:
- `ph.admin.groups.connect_instancetype('group_id','instancetype_id')`
- `ph.admin.groups.disconnect_instancetype('group_id','instancetype_id')`

In [ ]:
# Replace the instancetype's id below with yours
specified_instancetype_id = 'awesome-gpu'

# Connect an image to the group
ph.admin.groups.connect_instancetype(specified_group['id'], specified_instancetype_id)

# verify
ph.admin.groups.get(specified_group['id']['instanceTypes']) #instance"T"types, "T" uppercase.

In [ ]:
# Disconnect an image to the group
ph.admin.groups.disconnect_instancetype(specified_group['id'], specified_instancetype_id)

# verify
ph.admin.groups.get(specified_group['id'])['instanceTypes']

### Add/Remove Volume

Methods:
- `ph.admin.groups.connect_volume('group_id','volume_id')`
- `ph.admin.groups.disconnect_volume('group_id','volume_id')`

In [ ]:
# Replace the volume's id below with yours
specified_volume_id = 'example-dataset'

# Connect a volume to the group
ph.admin.groups.connect_volume(specified_group['id'], specified_volume_id)

# verify
ph.admin.groups.get(specified_group['id'])['volumes']

In [ ]:
# Disconnect an image to the group
ph.admin.groups.disconnect_volume(specified_group['id'], specified_volume_id)

# verify
ph.admin.groups.get(specified_group['id'])['volumes']

### Create an Image record and connect it to Group

Method `ph.admin.groups.create_image('group_id', image_configuration)`


e.g.

```python
image_config = {
  "name": "base",
  "displayName": "Base image",
  "description": "base-notebook with python 3.7",
  "type": "both",
  "url": "infuseai/docker-stacks:base-notebook-63fdf50a",
  "urlForGpu": "infuseai/docker-stacks:base-notebook-63fdf50a-gpu",
  "global": True
}
```

In [ ]:
image_config = {
  "name": "sdk-test-image",
  "displayName": "sdk test image",
  "description": "base-notebook with python 3.7",
  "type": "both",
  "url": "infuseai/docker-stacks:base-notebook-63fdf50a",
  "urlForGpu": "infuseai/docker-stacks:base-notebook-63fdf50a-gpu",
  "global": True
}

ph.admin.groups.create_image(specified_group['id'], image_config)

### Create an InstanceType and connect it to Group

Method: `ph.admin.groups.create_instanetype('group_id', instancetype_configuration)`

e.g.

```python
instancetype_config = {
  "name": "cpu-1",
  "displayName": "CPU 1",
  "description": "1 vCPU / 1G Memory",
  "cpuLimit": 1,
  "memoryLimit": 1,
  "gpuLimit": 0,
  "global": True,
  "tolerations": {
    "set": [
      {
        "operator": "Equal",
        "effect": "NoSchedule",
        "key": "nvidia.com/gpu",
        "value": "v100"
      }
    ]
  }
}
```

In [ ]:
instancetype_config = {
  "name": "sdk-test-cpu-1",
  "displayName": "Test CPU 1 by SDK",
  "description": "1 vCPU / 1G Memory",
  "cpuLimit": 1,
  "memoryLimit": 1,
  "gpuLimit": 0,
  "global": True,
  "tolerations": {
    "set": [
      {
        "operator": "Equal",
        "effect": "NoSchedule",
        "key": "nvidia.com/gpu",
        "value": "v100"
      }
    ]
  }
}
ph.admin.groups.create_instancetype(specified_group['id'], instancetype_config)

### Create a Volume and connect it to Group

Method: `ph.admin.groups.create_volume('group_id', writable_boolean, volume_configuration)`

e.g.

```python
volume_rec = {
  "name": "test-volume",
  "displayName": "test volume by SDK",
  "description": "desc",
  "type": "pv",
  "global": false,
  "pvProvisioning": "auto",
  "volumeSize": 1
}
```

In [ ]:
volume_config = {
  "name": "test-volume",
  "displayName": "test volume by SDK",
  "description": "desc",
  "type": "pv",
  "global": False,
  "pvProvisioning": "auto",
  "volumeSize": 1
}

ph.admin.groups.create_volume(specified_group['id'], True, volume_config)

### Update Group

Method `ph.admin.groups.update(user_id, config)`

By using `update()`, we can have a group sophiscated updates at once

Such as these changes as below in one update
- increase resources quotas
- enable the deployment feauture
- assign a user the group admin

Let's remove john-wick from *Continental* and revoke his administration priviledge (*Excommunicado*) in one update.

In [ ]:
group_name = 'Continental'
# Prepare a group configuration
updated_config = {
  "name": group_name,
  "enabledDeployment": True,
  "quotaCpu": 8,
  "quotaGpu": 8,
  "admins": [specified_user['id']]
#   "admins": ""  #remove all of admins
}

In [ ]:
result = ph.admin.groups.update(specified_group['id'], updated_config)
result

### Get Group Detail Info

Method `ph.admin.groups.get('group_id')`

Get the specified group detail by the group's id.

In [ ]:
# Get the group id by the given group name
group_name = 'Continental'
groups = ph.admin.groups.list()
for group in groups:
    if group['name'] == group_name:
        specified_group = group
        
ph.admin.groups.get(specified_group['id'])

### Delete Group

Delete the specified group by the given group's id.

In [ ]:
ph.admin.groups.delete(specified_group['id'])

## Common Operations <a class="anchor" id="common"></a>

In [ ]:
# Get the specified group detail by the given group name
group_name = 'Continental'
groups = ph.admin.groups.list()
for group in groups:
    if group['name'] == group_name:
        specified_group = group
        
group_detail = ph.admin.groups.get(specified_group['id'])
group_detail

### Get group admin users

In [ ]:
group_detail['admins']

### Get groups with Zero User

In [ ]:
groups = ph.admin.groups.list()
empty_groups = []

for group in groups:
    group_detail = ph.admin.groups.get(group['id'])
    if not group_detail['users']:
        empty_groups.append((group_detail['id'], group_detail['name']))
empty_groups    

### Get groups with running Deployments

In [ ]:
groups = ph.admin.groups.list()
deploymentsUsages = []

for group in groups:
    group_detail = ph.admin.groups.get(group['id'])
    if group_detail['deploymentsUsage']:
        deploymentsUsages.append((group_detail['name'], group_detail['deploymentsUsage']))
deploymentsUsages

### List groups with current resources usage

In [ ]:
groups = ph.admin.groups.list()
resourceStatus_by_group = []

for group in groups:
    group_detail = ph.admin.groups.get(group['id'])
    resourceStatus_by_group.append((group_detail['name'], group_detail['resourceStatus']))
resourceStatus_by_group

### List connected Images of a group

Method `ph.admin.groups.list_images('group_id')`

In [ ]:
ph.admin.groups.list_images(group_detail['id'])

### List connected InstanceTypes of a group

Method `ph.admin.groups.list_instancetypes('group_id')`

In [ ]:
ph.admin.groups.list_instancetypes(group_detail['id'])

### List connected Volumes of a group

Method `ph.admin.groups.list_volumes('group_id')`

In [ ]:
ph.admin.groups.list_volumes(group_detail['id'])

### Delete a specified group and Delete its members

In [ ]:
group_name = 'Continental'
groups = ph.admin.groups.list()
deleting_group = None

for group in groups:
    if group['name'] == group_name:
        deleting_group = group
        print(deleting_group)

if deleting_group:
    members = ph.admin.groups.list_users(deleting_group['id'])
    # Delete members if any
    for member in members:
        if member:
            ph.admin.users.delete(member['id'])
            print(f'Deleted {member["username"]}')

    # Finally, delete the group
    ph.admin.groups.delete(deleting_group['id'])